# Générateur de Recettes PDF

Ce notebook orchestre une collaboration multi-agents pour :
1. Collecter des contraintes utilisateur
2. Générer une recette personnalisée
3. Produire un PDF stylisé

**Agents** :  
- `InputCollector` → Collecte des préférences  
- `RecipeGenerator` → Crée la recette via LLM  
- `PDFGenerator` → Génère le document final

---

### Installation des bibliothèques

In [28]:
# Cell 1 - Installation
%pip install semantic-kernel python-dotenv reportlab --quiet

Note: you may need to restart the kernel to use updated packages.


## État Global & Configuration

In [29]:
class RecipeState:
    def __init__(self):
        self.diet: str = ""
        self.excluded_ingredients: list[str] = []
        self.guests: int = 4
        self.ingredients: list[str] = []
        self.steps: list[str] = []
        self.cooking_time: float = 0.0
        self.ready_to_generate: bool = False
        self.pdf_path: str = ""


### Plugins d'agents et fonctions de modification d'état

In [30]:
from semantic_kernel.functions import kernel_function
from reportlab.pdfgen import canvas

class InputCollectorPlugin:
    def __init__(self, state: RecipeState):
        self.state = state
    
    @kernel_function(name="set_diet", description="Définit le régime alimentaire")
    def set_diet(self, diet: str) -> str:
        self.state.diet = diet
        return f"Régime {diet} enregistré"

    @kernel_function(name="exclude_ingredient", description="Ajoute un ingrédient à exclure")
    def exclude_ingredient(self, ingredient: str) -> str:
        self.state.excluded_ingredients.append(ingredient)
        return f"Ingrédient {ingredient} exclu"

    @kernel_function(name="set_guests", description="Définit le nombre de convives")
    def set_guests(self, guests: int) -> str:
        self.state.guests = guests
        return f"{guests} convives prévus"

class RecipeGeneratorPlugin:
    def __init__(self, state: RecipeState):
        self.state = state

    @kernel_function(name="submit_recipe", description="Valide la recette générée")
    def submit_recipe(self, ingredients: list[str], steps: list[str], cooking_time: float) -> str:
        self.state.ingredients = ingredients
        self.state.steps = steps
        self.state.cooking_time = cooking_time
        self.state.ready_to_generate = True
        return "Recette validée et prête pour la génération PDF"



class PDFGeneratorPlugin:
    def __init__(self, state: RecipeState):
        self.state = state
    
    @kernel_function(name="generate_pdf", description="Génère le PDF final")
    def generate_pdf(self, output_path: str) -> str:
        c = canvas.Canvas(output_path)
        c.drawString(100, 800, "Recette personnalisée")
        c.drawString(100, 780, f"Pour {self.state.guests} personnes")
        c.save()
        self.state.pdf_path = output_path
        return f"PDF généré : {output_path}"


### Création des agents

In [31]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent

shared_state = RecipeState()

# InputCollector
input_kernel = Kernel()
input_kernel.add_plugin(InputCollectorPlugin(shared_state), "input_plugin")
input_agent = ChatCompletionAgent(
    kernel=input_kernel,
    name="InputCollector",
    instructions="""Collectez les informations utilisateur de manière structurée.
    - Demandez d'abord le régime alimentaire
    - Puis les ingrédients à exclure
    - Enfin le nombre de convives"""
)

# RecipeGenerator avec son plugin
recipe_kernel = Kernel()
recipe_kernel.add_plugin(RecipeGeneratorPlugin(shared_state), "recipe_plugin")
recipe_agent = ChatCompletionAgent(
    kernel=recipe_kernel,
    name="RecipeGenerator",
    instructions="""Générez des recettes en respectant les contraintes.
    - Convertir les ingrédients en liste Python
    - Structurer les étapes de préparation
    - Calculer le temps de cuisson"""
)


# PDFGenerator
pdf_kernel = Kernel()
pdf_kernel.add_plugin(PDFGeneratorPlugin(shared_state), "pdf_plugin")
pdf_agent = ChatCompletionAgent(
    kernel=pdf_kernel,
    name="PDFGenerator",
    instructions="""Générez un PDF professionnel:
    - Structurez en sections claires
    - Utilisez une mise en page aérée"""
)


### création de la conversation avec critères de terminaison et/ou de sélection

In [34]:
from semantic_kernel.agents import AgentGroupChat
from pydantic import PrivateAttr

from semantic_kernel.agents.strategies import TerminationStrategy

class ReadyTerminationStrategy(TerminationStrategy):
    def __init__(self, state: RecipeState):
        self._state = state

    async def should_agent_terminate(self, agent, history):
        return self._state.ready_to_generate


group_chat = AgentGroupChat(
    agents=[input_agent, recipe_agent, pdf_agent],
    termination_strategy=ReadyTerminationStrategy(shared_state),
    maximum_iterations=10
)


AttributeError: 'ReadyTerminationStrategy' object has no attribute '__pydantic_private__'

### Boucle principale

In [ ]:
async def recipe_workflow():
    recipe_query = (
        "Je souhaite une recette végétarienne pour 6 personnes, "
        "sans champignons ni produits laitiers."
    )
    
    group_chat.history.add_user_message(recipe_query)
    
    try:
        async for message in group_chat.invoke():
            print(f"[{message.name}] {message.content}")
            
            if shared_state.ready_to_generate:
                print("\\nTransition vers la génération PDF...")
                break
                
    except Exception as e:
        print(f"Erreur lors de la génération: {str(e)}")
    
    print("\\nProcessus terminé")

await recipe_workflow()


NameError: name 'group_chat' is not defined

### Génération de pdf

In [ ]:
if shared_state.pdf_path:
    print(f"PDF généré avec succès: {shared_state.pdf_path}")
    print("Contenu de la recette:")
    print(f"- Régime: {shared_state.diet}")
    print(f"- Ingrédients: {', '.join(shared_state.ingredients)}")
else:
    print("Échec de la génération:", 
          "Aucun PDF produit malgré les tentatives")
